# PMSM

> Definition of motor class using stator and rotor subclasses.

In [ ]:
#| default_exp pmsm

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
import sys
sys.path.append("..")

In [ ]:
#| export
import numpy as np
from PIL import Image
from emdesigner.stator import stator
from emdesigner.rotor import rotor

### motor class is created using the following:

1. stator class
2. rotor class
3. radial airgap

In [ ]:
0%3

0

In [ ]:
#| export
class pmsm(stator, rotor):
    def __init__(self, s = stator(), r = rotor()):
        self.stator = s
        self.rotor = r
        self.airgap = 0
        self.coilpitch = 1
        self.parallelpaths = 1
        self.valid = False
        self.params = 0
    
    def valid_design(self):
        if self.airgap <= 0:
            print('motor airgap cannot be negative or zero')
            return 0
        
        if self.coilpitch < 1:
            print('winding coil pitch cannot be smaller than 1 slots')
            return 0
        
        if self.parallelpaths < 1 or np.lcm(self.stator.slots, self.rotor.poles)%self.parallelpaths != 0:
            print('Invalid parallel paths for the slots-poles combination')
            return 0 
        
        # check validity of stator
        self.stator.valid_design()
        if self.stator.valid == False:
            print('stator validation pending')
            return 0

        # check validity of rotor
        self.rotor.valid_design()
        if self.rotor.valid == False:
            print('rotor validation pending')
            return 0

        # calculate motor parameters
        self.calculate_parameters()
        self.valid = True


    def calculate_parameters(self):
        """calculate parameters requied for calculation of motor properties
        """
        Nspp = self.stator.slots/(3*self.rotor.poles)
        
        # a_cp is modified to accommodate fractional slot concentrated winding
        a_cp = int(Nspp)/Nspp
        t_p = self.stator.params['Rsi']*self.rotor.params['th_p']
        t_s = self.stator.params['Rsi']*self.stator.params['th_s']
        t_c = self.stator.params['Rsi']*2*np.pi*self.coilpitch/self.stator.slots
        th_se = self.stator.params['th_s']*self.rotor.poles/2
        kd = np.sin(Nspp*th_se/2)/(Nspp*np.sin(th_se/2))
        a_m = self.rotor.poleembrace
        c_phi = 2*a_m/(1+a_m)
        gc = self.rotor.params['lm']/(self.airgap*c_phi)
        kc_1 = (t_s/self.stator.params['ws'])*(5*gc/self.stator.params['ws'] + 1)
        kc = 1/(1 - 1/kc_1)

        self.params = {'Nspp': Nspp,
                        'a_cp': a_cp,
                        't_p': t_p,
                        't_s': t_s,
                        't_c': t_c,
                        'th_se':th_se,
                        'kd': kd,
                        'c_phi': c_phi,
                        'gc': gc,
                        'kc':kc}


        # needs rotor information
        th_p = np.pi*2/self.poles
        Nspp = self.stator.slots/(3*self.rotor.poles)
        a_cp = int(Nspp)/Nspp
        t_p = self.stator.params['Rsi'] Rsi*th_p
        t_s = Rsi*th_s
        t_c = a_cp*t_p
        th_se = np.pi/(self.slots/self.poles)
        kd = np.sin(Nspp*th_se/2)/(Nspp*np.sin(th_se/2))
        lm = 4
        a_m = 0.83
        c_phi = 2*a_m/(1+a_m)
        gc = lm/(g*c_phi)
        kc_1 = (t_s/ws)*(5*gc/ws + 1)
        kc = 1/(1 - 1/kc_1) 

## test examples

define stator and rotor classes with corresponding values

In [ ]:
s1 = stator()
s1.slots = 12
s1.poles = 10
s1.outerdiameter = 125
s1.innerdiameter_fraction = 75/125
s1.slotopening_fraction = 0.3
s1.shoeheight_fraction = 0.1
s1.toothwidth_fraction = 0.5
s1.backiron_fraction = 0.3
s1.stacklength = 60
s1.windinglayers = 2
s1.coilpitch = 1
s1.turns = 11
s1.strands = 1
s1.strand_dia = 2
s1.parallelpaths = 2
s1.steel_grade = '50C470'

In [ ]:
s1.params

0

In [ ]:
s1.valid

False

In [ ]:
r1 = rotor()
r1.poles = 10
r1.outerdiameter = 73
r1.innerdiameter_fraction = 25/73
r1.poleembrace = 0.83
r1.magnetthickness_fraction = 2*4/(73 - 25)
r1.stacklength = 60
r1.magnetgrade = 'N30'
r1.steel_grade = '50C470'

In [ ]:
r1.params

0

### example 1:

create an instance of motor class using predefined stator and rotor classes

In [ ]:
m1 = pmsm(s1,r1)

In [ ]:
m1.rotor.outerdiameter

73

### example 2:

create an instance of motor class without any inputs. Assign a pre-existing stator and rotor class instances to the defined motor class instance

In [ ]:
m2 = pmsm()

In [ ]:
m2.rotor = r1
m2.stator = s1

In [ ]:
m2.rotor.outerdiameter

73

### example 3:

create an instance of motor class without any inputs. Assign stator and rotor values to the instance

In [ ]:
m3 = pmsm()

In [ ]:
m3.rotor.outerdiameter = 73

In [ ]:
m3.rotor.outerdiameter

73

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

## test examples for parameters calculation

In [ ]:
#m1.stator.valid_design()
#m1.rotor.valid_design()


In [ ]:
m1.airgap = 1
print(m1.stator.valid)


False


In [ ]:
m1.valid_design()

In [ ]:
m1.stator.params

{'Rso': 62.5,
 'Rsi': 37.5,
 'd1': 2.5,
 'd2': 2.5,
 'wbi': 6.0,
 'd3': 14.0,
 'th_s': 0.5235987755982988,
 'ws': 5.884432179588369,
 'wtb': 2.944486181930145,
 'wsi': 39.555513818069855,
 'wsb': 53.555513818069855,
 'As': 160.8155724712995}

In [ ]:
m1.rotor.params

{'Rro': 36.5,
 'Rri': 12.5,
 'lm': 4.0,
 'a_m': 0.83,
 'Am': 3066.445757315925,
 'th_p': 0.6283185307179586}

In [ ]:
m1.params

{'Nspp': 0.4,
 'a_cp': 0.0,
 't_p': 23.561944901923447,
 't_s': 19.634954084936204,
 't_c': 19.634954084936208,
 'th_se': 2.6179938779914944,
 'kd': 1.2940952255126037,
 'c_phi': 0.9071038251366119,
 'gc': 4.409638554216868,
 'kc': 1.0673891978270886}

In [ ]:
12/(3*10)

0.8

In [ ]:
the = np.pi*10/(12)

In [ ]:
np.sin(0.8*the/2)/(0.8*np.sin(the/2))

1.1207193402100668

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()